In [1]:
from sympy import *
from myst_nb import glue
from IPython.display import display, Math, HTML

G, GA, GAx, GAy, GAxy, GAz, GAxz = symbols('G, GA, GA_x, GA_y, GA_xy, GA_z, GA_xz')
E, EA,  EAx, EAy, EAz = symbols('E, EA,  EA_x, EA_y, EA_z')
GAxx, GAyy, GAzz = symbols('GA_{x}X, GA_{y}Y, GA_{z}Z')
EIxx, EIxy, EIyy, EIxz, EIzz, GJ = symbols('EI_xx, EI_xy, EI_yy, EI_xz, EI_zz, GJ')
Ixx, Ixy, Iyy, Ixz, Izz, J = symbols('I_xx, I_xy, I_yy, I_xz, I_zz, J')
σ_zz, σ_zx, σ_zy, σ_yz, σ_yx, σ_yy = symbols('σ_zz, σ_zx, σ_zy, σ_yz, σ_yx, σ_yy')
ε_zz, γ_zx, γ_zy, γ_yz, γ_yx, ε_yy = symbols('ε_zz, γ_zx, γ_zy, γ_yz, γ_yx, ε_yy')
u, v, w, x, y, z, A = symbols('u, v, w, x, y, z, A')
θx, θy, θz, θt, θ = symbols('θ_x, θ_y, θ_z, θ_t, θ')


# HAWC2
Manual: <br>
https://tools.windenergy.dtu.dk/HAWC2/downloads/13.1.0/manual/How2HAWC2.pdf

## Coordinate system: 
- orthogonal, forming right-handed coordinate system
- r is curved length of c/2 line, NOT straight span
- twist angle positive around z axis, sign oppposite to traditional notation for pitch angle 
    
<table>
<tr>
<td><img src="https://github.com/g-ang23/Blade-Elastic-Properties-Codes/tree/main/images/hawc2_intfoil_coo.png/hawc2_c2def_coo.jpeg"></td>
<td><img src="./images/hawc2_WT_coo.jpeg"></td>
</tr>
</table>   

## Structural properties
- cross sectional stiffness matrix is given @ elastic center rotated along the principal bending axes [p.25]
- twisted frame [p.43]
- centers given relative to c/2 [p.25]

### Stiffness matrix
<img src="./images/hawc2_intfoil_coo.png" width=30%></td>


In [6]:
stress = Matrix([[σ_zz, σ_zx, σ_zy]]).T
elastic_tensor = Matrix([[E,0,0], [0,G,0], [0,0,G]])
strain = Matrix([[ε_zz, γ_zx, γ_zy]]).T

'''analytical expression of stress'''
stress_anal = expand(      # expand calculations
elastic_tensor@strain      # matrix mul
.subs({                    # substitute
ε_zz: w + y*θx - x*θy, # i=0
γ_zx: u - y*θz,        # i=1
γ_zy: v + x*θz}))      # i=2

f_x = stress_anal[1] * A 
f_y = stress_anal[2] * A 
f_z = stress_anal[0] * A

m_x = stress_anal[0] * y * A
m_y = - stress_anal[0] * x * A
m_z = (stress_anal[2]*x - stress_anal[1]*y) * A


load = Matrix([f_x, f_y, f_z, m_x, m_y, m_z])
dofs = [u, v, w, θx, θy, θz]

K_mat = Matrix.zeros(6, 6)  # 6x6 stiffness matrix

for i, expr in enumerate(load):
    expr_exp = expand(expr)
    for j, dof in enumerate(dofs):
        # collect coefficient of the current DoF
        coeff = expr_exp.coeff(dof)
        K_mat[i, j] = coeff

K_hawc2=K_mat.subs({
    E*A:EA, E*A*y:EAx, E*A*x:EAy, E*A*x**2:EIyy, E*A*y**2:EIxx, E*A*x*y:EIxy,
    G*A:GA, G*A*y:GAxx, G*A*x:GAyy, G*A*x**2+ G*A*y**2:GJ})

display(Math(r"K_{hawc2}="+latex(K_hawc2))) 

<IPython.core.display.Math object>

In [7]:
K_HAWC2 = Matrix([ [GAx, GAxy, 0, 0, 0, -GAxx],
                   [GAxy, GAy, 0, 0, 0, GAyy], 
                   [0, 0, EA,   EAx, -EAy, 0 ],
                   [0, 0, EAx,  EIxx, -EIxy, 0], 
                   [0, 0, -EAy, -EIxy, EIyy, 0], 
                   [-GAxx, GAyy, 0, 0, 0, GJ]   ])
display(Math(r"K_{HAWC2}="+latex(K_HAWC2))) 

<IPython.core.display.Math object>

# BeamDyn
Manual: <br>
https://www.nrel.gov/docs/libraries/wind-docs/beamdyn_manual.pdf?sfvrsn=b5f0a6e3_3

## Coordinate system: 
- orthogonal, forming right-handed coordinate system
- z is curved length of pitch axis, NOT straight span
    
<table>
<tr>
<td><img src="./images/BeamDyn_foil_coo.jpeg"></td>
<td><img src="./images/BeamDyn_WT_coo.jpeg"></td>
</tr>
</table>   

## Structural properties
- cross sectional stiffness matrix is given @ pitch axis, local system [p.16]
- twisted frame 
- Steiner theorem applied wrt HAWC2 data

### Stiffness matrix
<table>
<tr>
<td><img src="./images/beamdyn_intfoil_coo.png"></td>
<td><img src="./images/BeamDyn_stiff_mat.jpeg"></td>
</tr>
</table>   

In [8]:
stress = Matrix([[σ_zz, σ_zx, σ_zy]]).T
elastic_tensor = Matrix([[E,0,0], [0,G,0], [0,0,G]])
strain = Matrix([[ε_zz, γ_zx, γ_zy]]).T

'''analytical expression of stress'''
stress_anal = expand(      # expand calculations
elastic_tensor@strain      # matrix mul
.subs({                    # substitute
ε_zz: w + y*θx - x*θy, # i=0
γ_zx: u - y*θz,        # i=1
γ_zy: v + x*θz}))      # i=2

f_x = stress_anal[1] * A 
f_y = stress_anal[2] * A 
f_z = stress_anal[0] * A

m_x = stress_anal[0] * y * A
m_y = - stress_anal[0] * x * A
m_z = (stress_anal[2]*x - stress_anal[1]*y) * A


load = Matrix([f_x, f_y, f_z, m_x, m_y, m_z])
dofs = [u, v, w, θx, θy, θz]

K_mat = Matrix.zeros(6, 6)  # 6x6 stiffness matrix

for i, expr in enumerate(load):
    expr_exp = expand(expr)
    for j, dof in enumerate(dofs):
        # collect coefficient of the current DoF
        coeff = expr_exp.coeff(dof)
        K_mat[i, j] = coeff

K_beamdyn=K_mat.subs({
    E*A:EA, E*A*y:EAx, E*A*x:EAy, E*A*x**2:EIyy, E*A*y**2:EIxx, E*A*x*y:EIxy,
    G*A:GA, G*A*y:GAxx, G*A*x:GAyy, G*A*x**2+ G*A*y**2:GJ})

display(Math(r"K_{BeamDyn}="+latex(K_beamdyn))) 

<IPython.core.display.Math object>

# hGAST
Thesis: <br>
https://freader.ekt.gr/eadd/index.php?doc=37152&lang=el

## Coordinate system: 
- orthogonal, forming right-handed coordinate system
- y is curved length of pitch axis, NOT straight span
- twist angle sign follows with traditional notation for pitch angle (towards wind)
    
<table>
<tr>
<td><img src="./images/hgast_foil_coo.jpeg"></td>
<td><img src="./images/hgast_WT_coo.jpeg"></td>
</tr>
</table>   

## Structural properties
- cross sectional stiffness matrix is given @ pitch axis, local system
- twisted frame 


### Stiffness matrix
- BeamDyn **"x"** $\rightarrow$ hGAST **"z"**
- BeamDyn **"y"** $\rightarrow$  hGAST **"x"**
<table>
<tr>
<td style="text-align:center;">
  <strong>hGAST</strong><br>
  <img src="./images/hgast_intfoil_coo.png" width="70%">
</td>
<td style="text-align:center;">
  <strong>BeamDyn</strong><br>
  <img src="./images/beamdyn_intfoil_coo.png" width="70%">
</td>
</tr>
</table>

In [9]:
K_hgast = Matrix([ [GAx, 0, GAxz,  0,  GAxx, 0 ],
                   [0, EA, 0,   -EAx, 0,    EAz], 
                   [GAxz, 0, GAz,  0,   -GAzz, 0],
                   [0, -EAx, 0,  EIxx, 0, -EIxz], 
                   [GAxx, 0, -GAzz, 0,    GJ,   0], 
                   [0, EAz, 0,  -EIxz, 0,   EIzz]])
display(Math(r"K_{hGAST}="+latex(K_hgast)))
display(Math(r"K_{BeamDyn}="+latex(K_beamdyn))) 

<IPython.core.display.Math object>

<IPython.core.display.Math object>